In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'open-iit-data-analytics-2023-kgp:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3859762%2F6694671%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240301%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240301T205325Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da649f21e308c5da6d785f79e3201631b649e95dcc41c1c82a8b417edbe3aff6c49c27841508dd08e4faf3c2f12ffc5b9e0d7591b4d8134169d1c6f0e0a500cf038c4676d19157d8712f0c13f375ebe9c304d964c28cddb9e3e22571e72bc19b41cb3ceed9a3e7df1ebf467ac99758fb2ffdb26c3089712d54d46ff45c281d885c65ad19b761243eb2e1e66583306e67aa4806b827264ef1450d1cbd4d1f7bb561cf29afe3954d9b0933852faf5847e8bfc5d207317d25721c1916ef943d1a78671e07a9876a610cd8ea01fec03415feffeb9e6edd11f5413089a1574769853c1c0daeadf0f6dd467bc485a1a41763b9fde076b60bb7d0d813c9436d5d0b24585'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# Imports
# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as PCA
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
register_matplotlib_converters()
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa import seasonal

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def parser(s):
    return datetime.strptime(s, '%d-%m-%Y')

def plot_time_graph(a = None , b = None , title = "Graph" , y = 'Freq'):
    plt.figure(figsize=(40,20))
    plt.plot(a)
    if (b != None):
      plt.plot(b)
    plt.title(title, fontsize=32)
    plt.ylabel(y, fontsize=24)
    start_date = datetime(2010,1,1)
    end_date = datetime(2023,12,1)
    for year in range(start_date.year,end_date.year):
        plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)

start_date = datetime(2010,1,1)
end_date = datetime(2022,11,1)

df = pd.read_csv('/kaggle/input/open-iit-data-analytics-2023-kgp/Automatic_extraction.csv', parse_dates=[0], index_col=0, date_parser=parser)
df2 = pd.read_csv('/kaggle/input/open-iit-data-analytics-2023-kgp/Manual_extraction.csv', parse_dates=[0], index_col=0, date_parser=parser)
tourist = pd.DataFrame(df['No of Tourist']).set_index(df.index)
df = df.dropna()


In [ ]:
# Chosing  tourist in data
tourist = tourist.asfreq(pd.infer_freq(tourist.index))
lim_tourist = tourist[start_date:end_date]
lim_df = df[start_date:end_date]
first_diff = tourist.diff()[1:]

# Ploting Shimla Tourist

In [ ]:
# plot_time_graph(a = tourist , title = 'Tourist Footfall per Month')
plt.figure(figsize=(40,20))
plt.plot(tourist)
# if (b != None):
#   plt.plot(b)
plt.title('Footfall of Tourist in Shimla', fontsize= 64)
plt.ylabel('No of Tourists', fontsize=40)
plt.xlabel('Date', fontsize=40)
start_date = datetime(2010,1,1)
end_date = datetime(2023,12,1)
for year in range(start_date.year,end_date.year):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
    # plt.text(pd.to_datetime(str(year)+'-01-01'),4,pd.to_datetime(str(year)+'-01-01'), fontsize=12)


# Plotting First Difference of No. of tourist

In [ ]:
# plot_time_graph(a = tourist , title = 'Tourist Footfall per Month')
plt.figure(figsize=(40,16))
plt.plot(first_diff)
# if (b != None):
#   plt.plot(b)
plt.title('First Difference in footfall of Tourist in Shimla', fontsize= 64)
plt.ylabel('First Difference', fontsize=40)
plt.xlabel('Date', fontsize=40)
start_date = datetime(2010,1,1)
end_date = datetime(2023,12,1)
for year in range(start_date.year,end_date.year):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)

# Plotting Covarience Matrix Of Trends.

Seosonality Decomposition

In [ ]:
from statsmodels.tsa import seasonal
decompose_data = seasonal.seasonal_decompose(tourist, model="additive")
decompose_data.plot();

# Automated

PCA

In [ ]:
# Model Application
pca=PCA(n_components=1)
reduced_data = pca.fit_transform(df.drop(columns = 'No of Tourist'))
reduced_data = pd.DataFrame(reduced_data)

# Data to CSV
# reduced_data.to_csv('pca2.csv')


#convert index column to datetime format
reduced_data.index = pd.to_datetime(df.index)
pd.concat([reduced_data,df['No of Tourist']], axis=1)

Code for PCA Graphs

In [ ]:
## Code for Corr matrix
pd.concat([reduced_data,df['No of Tourist']], axis=1)
matrix = reduced_data.corr(method = 'spearman')
plt.figure(figsize=(4,3))
plt.title(('PCA Dimensionality Reduction to features'), fontsize= 10)
plt.xlabel('Covariance')

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

_ = sns.heatmap(matrix, center=0, annot=True,
                fmt='.2f', square=True, cmap=cmap)



# Explained Varience Graph
plt.figure(figsize=(4,3))
plt.bar(range(1,len(pca.explained_variance_ )+1),pca.explained_variance_ )
plt.title(('Explained Varience vs Components features'), fontsize= 10)
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1,len(pca.explained_variance_ )+1),
        np.cumsum(pca.explained_variance_),
        c='red',
        label="Cumulative Explained Variance")
plt.legend(loc='upper left')

**Sarimax Model**


with covid data

In [ ]:
train_end = datetime(2020,6,1)
test_end = datetime(2022,12,1)
lim_tourist = tourist[start_date:test_end]
lim_df = reduced_data[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables

my_order = (3,1,1)
my_seasonal_order = (0, 0, 0, 0)
print('(p,d,q) = ', my_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values ) / ((test_data.values + predictions.values)/2))) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(30,12))

plt.plot(test_data)
plt.plot(predictions)
plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictions in Automatically extracted trends with Covid Data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year+1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(20,8))
plt.plot(residuals)
plt.title((('Residuals in Automatically extracted trends with Covid Data')), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()




Without Covid Data

In [ ]:
train_end = datetime(2017,12,1)
test_end = datetime(2020,1,1)
lim_tourist = tourist[start_date:test_end]
lim_df = reduced_data[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables

my_order = (3,1,1)
my_seasonal_order = (1, 0, 1, 12)
print('(p,d,q) = ', my_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values) / ((test_data.values + predictions.values)/2))) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(30,12))

plt.plot(test_data)
plt.plot(predictions)
plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictionsin Automatically extracted trends without Covid Data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year+1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(20,8))
plt.plot(residuals)
plt.title((('Residuals in order Automatically extracted features without covid data')), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()



# Semi automatic

In [ ]:
data = {
    'Tourism': [],
    'Traffic': [],
    'Lodging': []
}
fsa = pd.DataFrame(data)
tourism = df.loc[:, ['shimla tourism','himachal tourism','manali tourism','shimla tourism places','delhi tourism','hotel in shimla.3','shimla tourism packages','manali tourism places','himachal tourism hotel shimla','shimla hotels.3','himachal tourism.1','manali tourism.1','shimla tourism places.1','delhi tourism.1','hotel in shimla.1.1','shimla tourism packages.1','manali tourism places.1','himachal tourism hotel shimla.1','shimla hotels.1.1','himachal tourism.2','manali tourism.2','shimla tourism places.2','delhi tourism.2','hotel in shimla.2.1','shimla tourism packages.2','manali tourism places.2','himachal tourism hotel shimla.2']]
travel = df.loc[:, ['shimla hotels.2.1','shimla india','tour and travel','shimla hotel.3','shimla india.1','tour and travel.1','shimla hotel.1','shimla india.2','tour and travel.2','shimla hotel.2']]
lodging = df.loc[:, ['shimla hotel', 'hotel in shimla', 'manali hotel', 'shimla hotels', 'hotels in shimla', 'hotel in manali', 'resort in shimla', 'hotel in shimla.1', 'manali hotel.1', 'shimla hotels.1', 'hotels in shimla.1', 'hotel in manali.1', 'resort in shimla.1', 'hotel in shimla.2', 'manali hotel.2', 'shimla hotels.2', 'hotels in shimla.2', 'hotel in manali.2', 'resort in shimla.2']]
tourism = pd.DataFrame(tourism)
travel = pd.DataFrame(travel)
lodging = pd.DataFrame(lodging)
# tourism = df['shimla tourism','himachal tourism','manali tourism','shimla tourism places','delhi tourism','hotel in shimla.3','shimla tourism packages','manali tourism places','himachal tourism hotel shimla','shimla hotels.3','himachal tourism.1','manali tourism.1','shimla tourism places.1','delhi tourism.1','hotel in shimla.1.1','shimla tourism packages.1','manali tourism places.1','himachal tourism hotel shimla.1','shimla hotels.1.1','himachal tourism.2','manali tourism.2','shimla tourism places.2','delhi tourism.2','hotel in shimla.2.1','shimla tourism packages.2','manali tourism places.2','himachal tourism hotel shimla.2']
# travel = df['shimla hotels.2.1','shimla india','tour and travel','shimla hotel.3','shimla india.1','tour and travel.1','shimla hotel.1','shimla india.2','tour and travel.2','shimla hotel.2']
# lodging = df['shimla hotel', 'hotel in shimla', 'manali hotel', 'shimla hotels', 'hotels in shimla', 'hotel in manali', 'resort in shimla', 'hotel in shimla.1', 'manali hotel.1', 'shimla hotels.1', 'hotels in shimla.1', 'hotel in manali.1', 'resort in shimla.1', 'hotel in shimla.2', 'manali hotel.2', 'shimla hotels.2', 'hotels in shimla.2', 'hotel in manali.2', 'resort in shimla.2']



In [ ]:
# Model Application
pca=PCA(n_components=1)
tourism = pca.fit_transform(tourism)
tourism = pd.DataFrame(tourism)
travel = pca.fit_transform(travel)
travel = pd.DataFrame(travel)
lodging = pca.fit_transform(lodging)
lodging = pd.DataFrame(lodging)
#convert index column to datetime format
tourism.index = pd.to_datetime(df.index)
travel.index = pd.to_datetime(df.index)
lodging.index = pd.to_datetime(df.index)
fsa = pd.concat([tourism,travel,lodging,df['No of Tourist']], axis=1)

In [ ]:
## Code for Corr matrix
matrix = fsa.corr(method = 'spearman')
plt.figure(figsize=(4,3))
plt.title(('PCA Dimensionality Reduction to  features'), fontsize= 32)
plt.xlabel('Covariance')

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

_ = sns.heatmap(matrix, center=0, annot=True,
                fmt='.2f', square=True, cmap=cmap)



# Explained Varience Graph
plt.figure(figsize=(4,3))
plt.bar(range(1,len(pca.explained_variance_ )+1),pca.explained_variance_ )
plt.title(('Explained Varience vs Components features'), fontsize= 32)
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1,len(pca.explained_variance_ )+1),
        np.cumsum(pca.explained_variance_),
        c='red',
        label="Cumulative Explained Variance")
plt.legend(loc='upper left')

with covid data

In [ ]:
train_end = datetime(2020,6,1)
test_end = datetime(2022,12,1)
lim_tourist = tourist[start_date:test_end]
lim_df = fsa.drop(columns = 'No of Tourist')[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables

my_order = (3,1,1)
my_seasonal_order = (0, 0, 0, 0)
print('(p,d,q) = ', my_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values) / ((test_data.values + predictions.values)/2))) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(30,12))

plt.plot(test_data)
plt.plot(predictions)
plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictions in Semi-automatically extracted trends with Covid Data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year+1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(20,8))
plt.plot(residuals)
plt.title(('Residuals in Semi-automatically extracted trends with Covid Data'), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()




Without Covid Data

In [ ]:
train_end = datetime(2017,12,1)
test_end = datetime(2020,1,1)
lim_tourist = tourist[start_date:test_end]
lim_df = fsa.drop(columns = 'No of Tourist')[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables

my_order = (3,1,1)
my_seasonal_order = (1, 0, 1, 12)
print('(p,d,q) = ', my_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values ) / ((test_data.values + predictions.values)/2))) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(30,12))

plt.plot(test_data)
plt.plot(predictions)
plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictions in Semi-automatically extracted trends without Covid Data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year+1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(20,8))
plt.plot(residuals)
plt.title(('Residuals in Semi-automatically extracted trends without Covid Data'), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()





# Manual

In [ ]:
data = {
    'Tourism': [],
    'Traffic': [],
    'Recreation': [],
    'Lodging':[],
    'Dining':[],
    'Shopping': []
}
features = pd.DataFrame(data)
features['Tourism'] = df2['manali sightseeing'] + df2['shimla images']+df2['Shimla Sightseeing places'] +df2['Shimla Hill Station: (India)'] +df2['shimla railway station: (India)']+df2['Shimla places to visit: (India)']+df2['Kalka Shimla Railway: (India)']+df2['shimla tourist places: (India)']+ df2['shimla trip: (India)']




features['Traffic'] =  df2['toy train to shimla'] + +df2['shimla manali'] +df2['delhi to shimla distance: (India)']+df2['Shimla distance from Delhi: (India)'] + df2['Delhi to Shimla by road: (India)']

features['Recreation'] = df2['best time to visit shimla']  + df2['places to visit shimla'] + df2['Sightseeing Shimla: (India)']+df2['shimla sightseeing: (India)'] + df2['Shimla places to visit: (India)'] +df2['summer hill: (India)']

features['Lodging'] = df2['kolkata to shimla: (India)'] + + df2['delhi to shimla']

features['Dining'] =df2['Shimla Package: (India)'] + +df2['hotel in shimla mall road: (India)']

features['Shopping'] = df2['shimla shopping: (India)']





with covid data

In [ ]:
train_end = datetime(2020,6,1)
test_end = datetime(2022,12,1)
lim_tourist = tourist[start_date:test_end]
lim_df = features[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables

my_order = (4,1,1)
my_seasonal_order = (0, 0, 0, 0)
print('(p,d,q) = ', my_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values) / ((test_data.values + predictions.values)/2))) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(30,12))

plt.plot(tourist)
plt.plot(predictions)
# plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictions in Manual extracted features with covid data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year+1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(20,8))
plt.plot(residuals)
plt.title(('Residuals in Manual extracted features with covid data'), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()




Without Covid Data

In [ ]:
train_end = datetime(2017,12,1)
test_end = datetime(2020,1,1)
lim_tourist = tourist[start_date:test_end]
lim_df = features[start_date:test_end]

train_data = lim_tourist[:train_end]
test_data = lim_tourist[train_end + timedelta(days=1):test_end]
# test_data = pd.DataFrame(test_data, index=pd.date_range(start=(train_end + timedelta(days=1)).strftime('%m-%d-%Y'), end=test_end.strftime('%m-%d-%Y'), freq='MS'))

xtrain_data = (lim_df[:len(train_data)])
xtest_data = (lim_df[len(train_data):])


# To ignore all warnings
import warnings
warnings.filterwarnings("ignore")

# My Variables
my_order = (3,1,1)
my_seasonal_order = (1, 1, 2, 12)
print('(p,d,q) = ', my_order , my_seasonal_order)


#Define Model
model = SARIMAX(train_data, order=my_order, seasonal_order=my_seasonal_order, exog=xtrain_data)

# Time to implement model and Fit Model
start = time()
model_fit = model.fit()
end = time()
print('Model Fitting Time:', end - start)
# print(model_fit.summary())

#Getting Predictions
predictions_ci = (model_fit.get_forecast(len(test_data), exog = xtest_data)).conf_int()
predictions = model_fit.forecast(len(test_data), exog = xtest_data)
predictions = pd.Series(predictions, index=test_data.index)
predictions = pd.DataFrame(predictions)

#Getting residuals
residuals= test_data['No of Tourist'] - predictions['predicted_mean']
residuals
residuals = pd.DataFrame(residuals)



#Getting Error in Data
rmse = math.sqrt(mean_squared_error(test_data , predictions))
print('RMSE =',rmse )

# Calculate MAE
mae = mean_absolute_error(test_data , predictions)
print("Mean Absolute Error:", mae)

# Calculate MAPE and MASE
mape = np.mean(np.abs((test_data.values - predictions.values)) / ((test_data.values + predictions.values)/2)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape, "%")
seasonal_naive = test_data['No of Tourist'].diff(12).abs().mean()
mase = mae / (seasonal_naive if seasonal_naive else mae)
print("Mean Absolute Scaled Error (MASE):", mase)


#Getting Trends Matching Data
first_diff_pred = predictions.diff()[1:]
first_diff_test = test_data.diff()[1:]
e = pd.DataFrame(dtype = 'int64')

first_diff_pred = np.sign(np.sign(first_diff_pred['predicted_mean']))
first_diff_test = np.sign(np.sign(first_diff_test['No of Tourist']))
e =  first_diff_test * first_diff_pred
print(f"Percentage Matched: {((e > 0).mean()) * 100}")


# Plotting Predictions

plt.figure(figsize=(15,6))
# plt.plot(test_data)
plt.plot(lim_tourist)
plt.plot(predictions)
# plt.fill_between(predictions_ci.index, predictions_ci.iloc[:, 0], predictions_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.legend(('Actual', 'Predictions'), fontsize=12)

plt.title('Tourists Predictions Manual extracted features without covid data', fontsize=20)
plt.ylabel('No. of tourist', fontsize=16)
plt.xlabel('Date')
for year in range(2011,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()

# Plotting residuals
plt.figure(figsize=(12,5))
plt.plot(residuals)
plt.title(('Residuals in Manual extracted features without covid data'), fontsize=20)
plt.ylabel('Residual', fontsize=16)
plt.xlabel('Date')
for year in range(train_end.year +1,test_end.year+1):
    plt.axvline(pd.to_datetime(str(year)+'-01-01'), color='k', linestyle='--', alpha=0.2)
plt.show()



